# Structured Data Evaluation Harness

This notebook provides an interactive interface for evaluating generated structured data against ground truth data.

## Setup

1. Ensure your data files are ready:
   - `generated_data.xlsx` or `generated_data.csv` - Your generated data
   - `ground_truth.xlsx` or `ground_truth.csv` - Your ground truth data

2. Configure the paths and column mappings below.

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import json

from eval_harness.evaluator import StructuredDataEvaluator
from utils.create_test_data import create_test_data_files

#### Check Root Dir

In [ ]:
# Get the project root directory, this assumes the notebook is in eval_harness/ folder
current_dir = Path.cwd()

load_dotenv()

# Check if we're in eval_harness folder, if so go up one level
if current_dir.name == 'eval_harness':
    PROJECT_ROOT = current_dir.parent
else:
    # Otherwise assume we're in project root
    PROJECT_ROOT = os.getenv("PROJECT_ROOT")

print(f"Project root: {PROJECT_ROOT}")
print(f"Current working directory: {Path.cwd()}")

In [ ]:
# Load environment variables (from project root)
env_path = PROJECT_ROOT / ".env"
load_dotenv(env_path)

# Configuration - Option to create test data files
CREATE_TEST_DATA = True  # Set to True to generate test data files in the data/ folder

# Create test data files if requested
if CREATE_TEST_DATA:
    print("Creating test data files...")
    try:
        gen_path, gt_path = create_test_data_files(str(PROJECT_ROOT / "data"))
        print(f"\nTest data files created successfully!")
        print(f"  Generated data: {gen_path}")
        print(f"  Ground truth: {gt_path}")
    except Exception as e:
        print(f"Error creating test data files: {e}")
        print("Continuing with existing data files...")

# Configuration - Get paths from environment or use defaults
# Paths are resolved relative to project root
generated_path_env = os.getenv("GENERATED_DATA_PATH", "data/generated_data.xlsx")
ground_truth_path_env = os.getenv("GROUND_TRUTH_PATH", "data/ground_truth.xlsx")

# Resolve paths - if absolute, use as-is; if relative, resolve from project root
if os.path.isabs(generated_path_env):
    GENERATED_DATA_PATH = generated_path_env
else:
    GENERATED_DATA_PATH = str(PROJECT_ROOT / generated_path_env)

if os.path.isabs(ground_truth_path_env):
    GROUND_TRUTH_PATH = ground_truth_path_env
else:
    GROUND_TRUTH_PATH = str(PROJECT_ROOT / ground_truth_path_env)

# Verify files exist
print(f"Generated data path: {GENERATED_DATA_PATH}")
print(f"  File exists: {Path(GENERATED_DATA_PATH).exists()}")
print(f"Ground truth path: {GROUND_TRUTH_PATH}")
print(f"  File exists: {Path(GROUND_TRUTH_PATH).exists()}")

# Optional: Column mapping if column names differ
# Example: {"generated_col1": "ground_truth_col1", "generated_col2": "ground_truth_col2"}
COLUMN_MAPPING = None  # Set to dict if needed

# Match strategy: "index", "all_pairs", "truncate", or "key" (for key-based matching)
MATCH_STRATEGY = "key"  # Use "key" for matching rows by key column value

# Key column for matching rows (required when MATCH_STRATEGY="key")
# This column should exist in both datasets and contain values that can be matched
# (e.g., "key", "name", "id", "email", etc.)
KEY_COLUMN = "key"  # Change to your key column name

## Initialize Evaluator

In [ ]:
# Initialize the evaluator
evaluator = StructuredDataEvaluator(
    generated_data_path=GENERATED_DATA_PATH,
    ground_truth_path=GROUND_TRUTH_PATH,
    column_mapping=COLUMN_MAPPING,
    match_strategy=MATCH_STRATEGY,
    key_column=KEY_COLUMN if MATCH_STRATEGY == "key" else None,
)

print(f"Loaded {len(evaluator.generated_df)} rows from generated data")
print(f"Loaded {len(evaluator.ground_truth_df)} rows from ground truth data")
print(f"Evaluating {len(evaluator.column_names)} columns: {evaluator.column_names}")
if MATCH_STRATEGY == "key" and evaluator.matched_pairs is not None:
    matched_count = sum(1 for _, gt_idx, _ in evaluator.matched_pairs if gt_idx is not None)
    print(f"Matched {matched_count} out of {len(evaluator.matched_pairs)} rows using key column '{KEY_COLUMN}'")

## Preview Data

In [ ]:
# Preview generated data
print("\n=== Generated Data Preview ===")
display(evaluator.generated_df.head())

# Preview ground truth data
print("\n=== Ground Truth Data Preview ===")
display(evaluator.ground_truth_df.head())

## Evaluate All Columns

This section evaluates all columns using NLP distance and similarity metrics. When using key-based matching (MATCH_STRATEGY="key"), each row in the generated dataset is matched with the corresponding row in the ground truth dataset based on the key column value. The metrics shown below represent the **average** of comparisons between matched rows for each column.

Below is a description of each metric and how it's calculated:

### Distance and Similarity Metrics

**Cosine Similarity** (`cosine_similarity` / `cosine_similarity_tfidf`)
- **Range**: 0 to 1 (higher = more similar)
- **Description**: Measures the cosine of the angle between two text vectors in a high-dimensional space. Both metrics convert texts into numerical vectors and calculate the cosine of the angle between them, but they differ in how words are weighted:
  
  - **`cosine_similarity`** (TF only): Uses **Term Frequency (TF)** vectorization. Words are weighted only by how often they appear in each text. Common words like "the", "and", "of" have the same weight as rare, meaningful words. This is simpler and treats all words equally based on frequency.
  
  - **`cosine_similarity_tfidf`** (Full TF-IDF): Uses **Term Frequency-Inverse Document Frequency (TF-IDF)** vectorization. Words are weighted by both their frequency in the text (TF) and their rarity across the corpus (IDF). Common words are downweighted, while rare, distinctive words are upweighted. This better captures semantic importance and is more robust for comparing texts with different lengths or styles.
  
  **When to use which**: 
  - Use `cosine_similarity` (TF) when you want simple word frequency matching without penalizing common words
  - Use `cosine_similarity_tfidf` when you want to emphasize distinctive, meaningful words and reduce the impact of common stop words

- **Derivation**: `cosine_similarity = (A · B) / (||A|| × ||B||)` where A and B are text vectors (TF or TF-IDF weighted)

**Levenshtein Distance** (`levenshtein_distance`)
- **Range**: 0 to ∞ (lower = more similar)
- **Description**: Also known as edit distance, this measures the minimum number of single-character edits (insertions, deletions, or substitutions) required to transform one string into another. Lower values indicate more similar texts.
- **Derivation**: Calculated using dynamic programming to find the minimum edit operations needed

**Edit Distance** (`edit_distance` / `edit_distance_normalized`)
- **Range**: 
  - Raw: 0 to ∞ (lower = more similar)
  - Normalized: 0 to 1 (lower = more similar)
- **Description**: Same as Levenshtein distance, but the normalized version divides by the maximum length of the two strings to provide a scale-independent measure between 0 and 1.
- **Derivation**: `normalized_edit_distance = levenshtein_distance / max(len(text1), len(text2))`

**Jaccard Similarity** (`jaccard_similarity_unigram` / `jaccard_similarity_bigram`)
- **Range**: 0 to 1 (higher = more similar)
- **Description**: Measures similarity as the size of the intersection divided by the size of the union of two sets. 
  - **Unigram**: Compares sets of individual words
  - **Bigram**: Compares sets of character pairs (2-character sequences)
- **Derivation**: `jaccard_similarity = |A ∩ B| / |A ∪ B|` where A and B are sets of n-grams

### Interpretation Tips

- **High cosine similarity** (>0.8): Texts are semantically similar, even if worded differently
- **Low Levenshtein/edit distance**: Texts are character-by-character similar (good for detecting typos)
- **High Jaccard similarity**: Texts share many common words or character sequences
- **Combined metrics**: Using multiple metrics together provides a more robust evaluation, as each captures different aspects of text similarity

In [ ]:
# Evaluate all columns with all metrics
results = evaluator.evaluate_all_columns()

# Display summary report
summary_df = evaluator.get_summary_report(results)
display(summary_df)

## Evaluate Specific Column

In [ ]:
# Evaluate a specific column
column_name = evaluator.column_names[0]  # Change to your column name

column_results = evaluator.evaluate_column(column_name)

print(f"\n=== Results for column: {column_name} ===")
print(f"Number of comparisons: {column_results['num_comparisons']}")
print("\nMetrics:")
for metric_name, metric_stats in column_results.items():
    if metric_name not in ["column_name", "num_comparisons"]:
        if isinstance(metric_stats, dict):
            print(f"\n{metric_name}:")
            for stat_name, stat_value in metric_stats.items():
                print(f"  {stat_name}: {stat_value:.4f}")
        else:
            print(f"{metric_name}: {metric_stats}")

## Control Which Metrics You Evaluate With

In [ ]:
# Evaluate with only specific metrics
specific_metrics = ["cosine_similarity", "edit_distance", "jaccard_similarity"]

results_specific = evaluator.evaluate_all_columns(metrics=specific_metrics)
summary_specific = evaluator.get_summary_report(results_specific)

display(summary_specific)

## Detailed Analysis

In [ ]:
# Get detailed results
print("\n=== Detailed Results ===")
print(json.dumps(results, indent=2, default=str))

## Key-Matched Row Comparison Analysis

This section shows comparisons between matched rows based on the key column. Each row in the generated dataset is matched with the corresponding row in the ground truth dataset that has the same key value. The metrics shown are calculated only for these matched pairs, and column-level metrics represent the average across all matched pairs.

In [ ]:
# Select the column to analyze
# Change this to any column name from evaluator.column_names
SELECTED_COLUMN = "name"  # Options: name, email, address, phone, description

# Verify column exists
if SELECTED_COLUMN not in evaluator.column_names:
    print(f"Error: Column '{SELECTED_COLUMN}' not found.")
    print(f"Available columns: {evaluator.column_names}")
else:
    print(f"Selected column: {SELECTED_COLUMN}")
    print(f"Generated values: {len(evaluator.aligned_generated[SELECTED_COLUMN])}")
    print(f"Ground truth values: {len(evaluator.aligned_ground_truth[SELECTED_COLUMN])}")

In [ ]:
# Get matched comparisons for the selected column
if evaluator.match_strategy == "key" and SELECTED_COLUMN in evaluator.column_names:
    matched_df = evaluator.get_matched_comparisons(SELECTED_COLUMN)
    
    if len(matched_df) > 0:
        print(f"\n=== Matched Row Comparisons for Column: {SELECTED_COLUMN} ===")
        print(f"Total matched pairs: {len(matched_df)}\n")
        display(matched_df)
        
        # Calculate and display average metrics for this column
        print(f"\n=== Average Metrics for Column: {SELECTED_COLUMN} ===")
        metric_cols = [col for col in matched_df.columns if col not in 
                       ["key", "generated_value", "ground_truth_value"]]
        
        avg_metrics = {}
        for metric in metric_cols:
            avg_metrics[metric] = matched_df[metric].mean()
        
        avg_df = pd.DataFrame([avg_metrics]).T
        avg_df.columns = ["average"]
        avg_df = avg_df.sort_values("average", ascending=False)
        display(avg_df)
        
        print(f"\nNote: These averages represent the mean similarity/distance metrics")
        print(f"across all {len(matched_df)} matched rows (based on key column '{evaluator.key_column}').")
    else:
        print(f"No matched pairs found for column '{SELECTED_COLUMN}'.")
        print("This may indicate that no rows in the generated data matched with ground truth rows.")
else:
    print("Please ensure key-based matching is enabled and column is valid.")

## Export Results

In [ ]:
# Export summary to CSV in the data folder
output_path = PROJECT_ROOT / "data" / "evaluation_results.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)  # Ensure data folder exists
summary_df.to_csv(output_path, index=False)
print(f"Results exported to {output_path}")